In [1]:
from zad11 import connect, load_table, process

In [ ]:
spark = connect()

Connecting to Spark...


In [ ]:
load_table(spark, "/input/daily_weather_2017.csv")

In [ ]:
load_table(spark, "/input/cities.csv")

In [ ]:
process(spark, "daily_country_weather", """
SELECT
    c.country,
    w.date,
    AVG(w.temperature_c) as temperature_c,
    COALESCE(AVG(w.precipitation_mm), 0) as precipitation_mm
FROM (
    SELECT
        station_id,
        to_date(date) as date,
        avg_temp_c as temperature_c,
        precipitation_mm
    FROM daily_weather_2017
    WHERE date BETWEEN '2017-01-01' AND '2021-12-31'
) AS w
JOIN cities AS c
    ON w.station_id = c.station_id
GROUP BY c.country, w.date
HAVING temperature_c IS NOT NULL
ORDER BY c.country, w.date
""")

In [ ]:
spark.stop()